# AWS pandas SDK를 이용한 Dynamodb 사용
---

#### 라이브러리 준비

In [1]:
import awswrangler as wr
import boto3
import json
from decimal import Decimal

In [69]:
import warnings
warnings.filterwarnings('ignore')

#### 데이터 준비

In [17]:
# JSON 파일 읽기
with open('sample_orders.json', 'r') as file:
    orders = json.load(file)

In [ ]:
# Decimal 타입으로 변환하는 함수
def convert_to_decimal(obj):
    if isinstance(obj, dict):
        return {k: convert_to_decimal(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_decimal(v) for v in obj]
    elif isinstance(obj, float):
        return Decimal(str(obj))
    else:
        return obj

#### 데이터 삽입

In [25]:
# dynamodb는 실수형 숫자 타입 Float을 지원하지 않기 때문에 Decimal 타입으로 형변환합니다.
orders = [convert_to_decimal(order) for order in orders]

In [ ]:
# boto3 session 설정
boto3_session = boto3.Session(
    profile_name='awstutor',
    region_name='ap-northeast-2'
)

In [26]:
# 데이터 삽입
wr.dynamodb.put_items(
    table_name='Orders',
    items=orders,
    boto3_session=boto3_session
)

#### 데이터 조회

In [70]:
new_orders = wr.dynamodb.read_items(
    table_name='Orders',
    boto3_session=boto3_session,
    allow_full_scan=True  # 전체 테이블 스캔을 허용합니다.
)

In [30]:
new_orders

,Items,OrderStatus,TotalAmount,CustomerId,OrderId,OrderDate
0,"[{'id': 'P4', 'name': 'Backpack', 'price': 59....",Delivered,179.97,C0065,O-7309f2f2-c0b7-4001-95bf-8bd0a46f8e36,2023-01-01
1,"[{'id': 'P2', 'name': 'Jeans', 'price': 49.99,...",Pending,539.92,C0065,O-a3e8837a-cc21-4cb4-ad3f-4e9a6cb78dfd,2023-01-13
2,"[{'id': 'P3', 'name': 'Sneakers', 'price': 79....",Pending,329.92,C0065,O-6e9e1abe-ba91-4167-98f3-065e3dac4e8e,2023-01-31
3,"[{'id': 'P2', 'name': 'Jeans', 'price': 49.99,...",Delivered,49.99,C0065,O-324a3473-956b-4c80-9047-d9f9c7d70ea1,2023-02-02
4,"[{'id': 'P8', 'name': 'Sweater', 'price': 39.9...",Pending,129.96,C0065,O-a910678f-35fd-4aaa-8ecb-9fe14825d4e8,2023-02-24
...,...,...,...,...,...,...
986,"[{'id': 'P10', 'name': 'Skirt', 'price': 34.99...",Pending,474.92,C0039,O-69096241-021f-4fa5-8cd1-8c00aa4fe1db,2023-09-08
987,"[{'id': 'P7', 'name': 'Shorts', 'price': 29.99...",Delivered,524.91,C0039,O-ebaa7eaf-75a5-4cd1-a4ae-8fe9e2f4e7e0,2023-10-15
988,"[{'id': 'P3', 'name': 'Sneakers', 'price': 79....",Pending,79.99,C0039,O-1ab14c4f-423c-47ac-9d6f-e0333c3ce232,2023-11-02
989,"[{'id': 'P2', 'name': 'Jeans', 'price': 49.99,...",Pending,534.90,C0039,O-11323963-d8ad-4c41-bf31-10ef1e3611bc,2023-11-18


#### 데이터 필터링

In [31]:
# C0039 고객의 데이터만 조회 (wr.dynamodb 이용)
customer_c0039_orders = wr.dynamodb.read_items(
    table_name='Orders',
    boto3_session=boto3_session,
    filter_expression='CustomerId = :customer_id',
    expression_attribute_values={':customer_id': 'C0039'}
)

# 결과 출력
print("C0039 고객의 주문 데이터:")
print(customer_c0039_orders)

# 주문 수 출력
print(f"\nC0039 고객의 총 주문 수: {len(customer_c0039_orders)}")



C0039 고객의 주문 데이터:
                                                Items OrderStatus TotalAmount  \
0   [{'id': 'P3', 'name': 'Sneakers', 'price': 79....     Pending       79.99   
1   [{'id': 'P1', 'name': 'T-shirt', 'price': 19.9...   Delivered       59.97   
2   [{'id': 'P1', 'name': 'T-shirt', 'price': 19.9...   Delivered      539.88   
3   [{'id': 'P8', 'name': 'Sweater', 'price': 39.9...   Delivered      439.92   
4   [{'id': 'P7', 'name': 'Shorts', 'price': 29.99...     Pending      409.92   
5   [{'id': 'P1', 'name': 'T-shirt', 'price': 19.9...   Delivered      279.93   
6   [{'id': 'P10', 'name': 'Skirt', 'price': 34.99...     Pending      474.92   
7   [{'id': 'P7', 'name': 'Shorts', 'price': 29.99...   Delivered      524.91   
8   [{'id': 'P3', 'name': 'Sneakers', 'price': 79....     Pending       79.99   
9   [{'id': 'P2', 'name': 'Jeans', 'price': 49.99,...     Pending      534.90   
10  [{'id': 'P10', 'name': 'Skirt', 'price': 34.99...     Shipped       34.99   

   Custom

/Users/dante/aws-ds-class/.conda/lib/python3.12/site-packages/awswrangler/dynamodb/_read.py:411: UserWarning: Attempting DynamoDB Scan operation with arguments:
{'ConsistentRead': False, 'FilterExpression': 'CustomerId = :customer_id', 'ExpressionAttributeValues': {':customer_id': {'S': 'C0039'}}}
  warnings.warn(


#### 특정 사용자 데이터 삭제

In [67]:
for idx, row in customer_c0039_orders.iterrows():
    wr.dynamodb.delete_items(
        table_name='Orders',
    boto3_session=boto3_session,
    items=[{
        'OrderDate': row['OrderDate'],
        'CustomerId': row['CustomerId']
    }]
)

In [68]:
# 삭제 후 데이터 조회
wr.dynamodb.read_items(
    table_name='Orders',
    boto3_session=boto3_session,
    filter_expression='CustomerId = :customer_id',
    expression_attribute_values={':customer_id': 'C0039'}
)

/Users/dante/aws-ds-class/.conda/lib/python3.12/site-packages/awswrangler/dynamodb/_read.py:411: UserWarning: Attempting DynamoDB Scan operation with arguments:
{'ConsistentRead': False, 'FilterExpression': 'CustomerId = :customer_id', 'ExpressionAttributeValues': {':customer_id': {'S': 'C0039'}}}
  warnings.warn(


""


#### 테이블 삭제

In [76]:
order_table = wr.dynamodb.get_table(
    table_name='Orders',
    boto3_session=boto3_session
)
order_table

dynamodb.Table(name='Orders')

In [77]:
order_table.delete()

{'TableDescription': {'TableName': 'Orders',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ap-northeast-2:905418381372:table/Orders',
  'TableId': '53823d93-3df0-49a8-bd41-b949558328c1',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'UKMRNB23BK55SL1FI2FACPC1CBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 20 Jul 2024 06:17:49 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '353',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'UKMRNB23BK55SL1FI2FACPC1CBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3139304535'},
  'RetryAttempts': 0}}